# Strands 멀티 에이전트 시스템과 AgentCore Memory Tool (단기 메모리)

## 소개

이 노트북은 AWS AgentCore Memory와 Strands 프레임워크를 사용하여 **공유 메모리를 가진 멀티 에이전트 시스템**을 구현하는 방법을 보여줍니다. 이전 예제들이 단일 에이전트 메모리에 초점을 맞췄던 반면, 이 노트북은 여러 전문 에이전트가 공통 메모리 저장소에 액세스하면서 함께 작업하는 방법을 탐구합니다.

## 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 단기 대화형                                                                        |
| 에이전트 사용 사례     | 여행 계획 어시스턴트                                                            |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                   |
| 튜토리얼 구성요소     | AgentCore 단기 메모리, Strands Agents, 도구를 통한 메모리 검색           |
| 예제 복잡도          | 초급                                                                             |


배울 내용:

- 여러 에이전트가 액세스할 수 있는 공유 메모리 리소스 설정 방법
- 자체 메모리 액세스를 가진 도구로서 전문 에이전트 생성
- 전문 에이전트에게 위임하는 코디네이터 에이전트 구현
- 여러 에이전트 상호작용 간 대화 컨텍스트 유지

### 시나리오 컨텍스트

이 예제에서는 다음과 같은 **여행 계획 시스템**을 만들어보겠습니다:
1. 항공 여행 전문 항공편 예약 어시스턴트
2. 숙박 시설에 초점을 맞춘 호텔 예약 어시스턴트
3. 이러한 전문 에이전트들에게 위임하는 여행 코디네이터

이 접근 방식은 복잡한 도메인을 동일한 메모리 저장소를 공유하는 전문 에이전트로 나눠 구성할 수 있는 방법을 보여줍니다.

## 아키텍처
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 전제조건
- Python 3.10+
- 적절한 권한이 있는 AWS 계정
- AgentCore Memory에 대한 적절한 권한이 있는 AWS IAM 역할
- Amazon Bedrock 모델에 대한 액세스

환경 설정과 공유 메모리 리소스 생성을 시작해보겠습니다!

## 1단계: 환경 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent

Amazon Bedrock 모델과 AgentCore에 대한 적절한 권한을 가진 지역과 역할을 정의합니다

In [ ]:
import os
region = os.getenv('AWS_REGION', 'us-west-2')
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S")
logger = logging.getLogger("agentcore-memory")

## 2단계: 공유 메모리 생성
이 섹션에서는 전문 에이전트들 간에 공유될 메모리 리소스를 생성합니다.

In [ ]:
from bedrock_agentcore.memory import MemoryClient

In [ ]:
client = MemoryClient(region_name=region)
memory_name = "TravelAgent_STM_%s" % datetime.now().strftime("%Y%m%d%H%M%S")
memory_id = None


In [ ]:
from botocore.exceptions import ClientError

try:
    print("Creating Memory...")
    memory_name = memory_name

    # Create the memory resource
    memory = client.create_memory_and_wait(
        name=memory_name,                       # Unique name for this memory store
        description="Travel Agent STM",         # Human-readable description
        strategies=[],                          # No special memory strategies for short-term memory
        event_expiry_days=7,                    # Memories expire after 7 days
        max_wait=300,                           # Maximum time to wait for memory creation (5 minutes)
        poll_interval=10                        # Check status every 10 seconds
    )

    # Extract and print the memory ID
    memory_id = memory['id']
    print(f"Memory created successfully with ID: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    print(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()

    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

### 멀티 에이전트 시스템을 위한 공유 메모리 이해

우리가 생성한 메모리 리소스는 여행 계획 시스템의 공유 지식 기반 역할을 합니다. 모든 에이전트는 이 공통 메모리 저장소에서 읽고 쓰게 되어 다음을 가능하게 합니다:

1. **지식 일관성**: 모든 에이전트가 동일한 정보로 작업
2. **컨텍스트 보존**: 에이전트 전환 간에도 대화 기록 유지
3. **전문화된 액세스**: 각 에이전트는 고유한 actor_id를 가지지만 session_id는 공유

이 접근 방식은 전문 에이전트가 전체 대화 컨텍스트의 이점을 누리면서도 자신의 도메인에 집중할 수 있게 해줍니다.

## 3단계: 메모리 훅 프로바이더 생성

이 단계에서는 메모리 작업을 자동화하는 사용자 정의 `MemoryHookProvider` 클래스를 정의합니다. 훅은 에이전트의 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 우리가 만드는 메모리 훅은 두 가지 주요 기능을 제공합니다:

1. **메모리 검색**: 사용자가 메시지를 보낼 때 관련된 과거 대화를 자동으로 가져옴
2. **메모리 저장**: 에이전트가 응답한 후 새로운 대화를 저장

이는 수동 관리 없이 원활한 메모리 경험을 만들어줍니다.

In [ ]:
class ShortTermMemoryHook(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """Load recent conversation history when agent starts"""
        try:
            # Get last 5 conversation turns
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5,
                branch_name="main"
            )
            
            if recent_turns:
                # Format conversation history for context
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role'].lower()
                        content = message['content']['text']
                        context_messages.append(f"{role.title()}: {content}")
                
                context = "\n".join(context_messages)
                logger.info(f"Context from memory: {context}")
                
                # Add context to agent's system prompt
                event.agent.system_prompt += f"\n\nRecent conversation history:\n{context}\n\nContinue the conversation naturally based on this context."
                
                logger.info(f"✅ Loaded {len(recent_turns)} recent conversation turns")
            else:
                logger.info("No previous conversation history found")
                
        except Exception as e:
            logger.error(f"Failed to load conversation history: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """Store conversation turns in memory"""
        messages = event.agent.messages
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
            )
            
        except Exception as e:
            logger.error(f"Failed to store message: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        # Register memory hooks
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## 4단계: Strands Agents로 멀티 에이전트 아키텍처 생성
이 섹션에서는 항공편과 호텔 예약을 위한 전문 에이전트로 구성된 멀티 에이전트 시스템을 만들어보겠습니다. 두 에이전트 모두 메모리 리소스에 대한 액세스를 공유합니다.

In [ ]:
# Import the necessary components
from strands import Agent, tool

In [ ]:
# Create unique actor IDs for each specialized agent but share the session ID
flight_actor_id = f"flight-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
hotel_actor_id = f"hotel-user-{datetime.now().strftime('%Y%m%d%H%M%S')}"
session_id = f"travel-session-{datetime.now().strftime('%Y%m%d%H%M%S')}"
flight_namespace = f"travel/{flight_actor_id}/preferences"
hotel_namespace = f"travel/{hotel_actor_id}/preferences"

### 메모리 액세스를 가진 전문 에이전트 생성

다음으로, 전문 에이전트를 위한 시스템 프롬프트를 정의하겠습니다. 각 프롬프트에는 에이전트가 파싱할 수 있는 형식의 메모리 매개변수가 포함되어 있습니다:

In [ ]:
# System prompt for the hotel booking specialist
HOTEL_BOOKING_PROMPT = f"""You are a hotel booking assistant. Help customers find hotels, make reservations, and answer questions about accommodations and amenities. 
Provide clear information about availability, pricing, and booking procedures in a friendly, helpful manner."""

# System prompt for the flight booking specialist
FLIGHT_BOOKING_PROMPT = f"""You are a flight booking assistant. Help customers find flights, make reservations, and answer questions about airlines, routes, and travel policies. 
Provide clear information about flight availability, pricing, schedules, and booking procedures in a friendly, helpful manner."""

### 에이전트 도구 구현
이제 코디네이터 에이전트가 사용할 수 있는 도구로서 전문 에이전트를 구현해보겠습니다:

In [ ]:
@tool
def flight_booking_assistant(query: str) -> str:
    """
    Process and respond to flight booking queries.

    Args:
        query: A flight-related question about bookings, schedules, airlines, or travel policies

    Returns:
        Detailed flight information, booking options, or travel advice
    """
    try:
        flight_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            memory_client=client,
            actor_id=flight_actor_id,
            session_id=session_id
        )
        
        flight_agent = Agent(hooks=[flight_memory_hooks],model=MODEL_ID, system_prompt=FLIGHT_BOOKING_PROMPT)

        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight booking assistant: {str(e)}"

@tool
def hotel_booking_assistant(query: str) -> str:
    """
    Process and respond to hotel booking queries.

    Args:
        query: A hotel-related question about accommodations, amenities, or reservations

    Returns:
        Detailed hotel information, booking options, or accommodation advice
    """
    try:
        hotel_memory_hooks = ShortTermMemoryHook(
            memory_id=memory_id,
            memory_client=client,
            actor_id=hotel_actor_id,
            session_id=session_id
        )

        hotel_booking_agent = Agent(hooks=[hotel_memory_hooks],model=MODEL_ID, system_prompt=HOTEL_BOOKING_PROMPT)
        
        # Call the agent and return its response
        response = hotel_booking_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in hotel booking assistant: {str(e)}"

### 코디네이터 에이전트 생성

마지막으로, 이러한 전문 도구들 간에 조정을 담당하는 메인 여행 계획 에이전트를 만들어보겠습니다:

In [ ]:
# System prompt for the coordinator agent
TRAVEL_AGENT_SYSTEM_PROMPT = """
You are a comprehensive travel planning assistant that coordinates between specialized tools:
- For flight-related queries (bookings, schedules, airlines, routes) → Use the flight_booking_assistant tool
- For hotel-related queries (accommodations, amenities, reservations) → Use the hotel_booking_assistant tool
- For complete travel packages → Use both tools as needed to provide comprehensive information
- For general travel advice or simple travel questions → Answer directly

Each agent will have its own memory in case the user asks about historic data.
When handling complex travel requests, coordinate information from both tools to create a cohesive travel plan.
Provide clear organization when presenting information from multiple sources. \
Ask max two questions per turn. Keep the messages short, don't overwhelm the customer.
"""

In [ ]:
travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

#### 멀티 에이전트 시스템이 준비되었습니다!!

## 에이전트를 테스트해보겠습니다.

여행 계획 시나리오로 멀티 에이전트 시스템을 테스트해보겠습니다:

In [ ]:
response = travel_agent("Hello, I would like to book a trip from LA to Madrid. From July 1 to August 2.")

In [ ]:
response = travel_agent("I would only like to focus on the flight at the moment. direct flimid-range, city center, pool, standard room")

## 메모리 지속성 테스트

메모리 시스템이 올바르게 작동하는지 테스트하기 위해 여행 에이전트의 새 인스턴스를 생성하고 이전에 저장된 정보에 액세스할 수 있는지 확인해보겠습니다:

In [ ]:
# Create a new instance of the travel agent
new_travel_agent = Agent(
    system_prompt=TRAVEL_AGENT_SYSTEM_PROMPT,
    model=MODEL_ID,
    tools=[flight_booking_assistant, hotel_booking_assistant]
)

# Ask about previous conversations
new_travel_agent("Can you remind me about flights talked about before?")

## 요약

이 노트북에서 보여준 내용:

1. 여러 에이전트를 위한 공유 메모리 리소스 생성 방법
2. 메모리 액세스를 가진 도구로서 전문 에이전트 구현 방법
3. 대화 컨텍스트를 유지하면서 여러 에이전트 간 조정 방법
4. 다른 에이전트 인스턴스 간 메모리 지속 방법

공유 메모리를 가진 이 멀티 에이전트 아키텍처는 일관된 사용자 경험을 유지하면서 전문 도메인을 처리할 수 있는 복잡한 대화형 AI 시스템을 구축하는 강력한 접근 방식을 제공합니다.

## 정리
이 노트북에서 사용한 리소스를 정리하기 위해 메모리를 삭제해보겠습니다.

In [ ]:
#client.delete_memory_and_wait(
#        memory_id = memory_id,
#        max_wait = 300,
#        poll_interval =10
#)